In [1]:
import pandas as pd
import numpy as np
import pandas.io.sql as sqlio
import psycopg2
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from sklearn.feature_extraction.text import TfidfVectorizer
import datetime
from sklearn.cluster import KMeans
from sklearn import metrics
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.decomposition import TruncatedSVD
#https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html
#sklearn.cluster.MiniBatchKMeans
from sklearn.cluster import MiniBatchKMeans
stop = stopwords.words('english')
now = datetime.datetime.now()

In [2]:
conn = psycopg2.connect(user="xxxxx", password="xxxx", host="xxxx", database="Ingestion")

In [3]:
query = "select * from itc_obs_0421_savecheck3"
#execute query and save it to a variable
dataset = sqlio.read_sql_query(query,conn)

C:\Users\schre\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
pd.set_option('display.max_columns', None)
dataset.head()
dataset.shape

(528640, 39)

In [5]:
df = dataset[['dollars_obligated','description_of_requirement','level_2_category','level_3_category','co_bus_size_determination',
                   'business_rule_tier','contract_name']]

In [6]:
df = df.astype({'level_2_category': str})
df = df.astype({'level_3_category': str})
df = df.astype({'co_bus_size_determination': str})
df = df.astype({'contract_name': str})
df = df.astype({'level_3_category': str})
df = df.astype({'description_of_requirement': str})

In [7]:
df["req"] = df[['description_of_requirement','level_2_category','level_3_category','co_bus_size_determination',
                   'business_rule_tier','contract_name']].apply(lambda x: ' '.join(x), axis = 1)

In [8]:
df['req'].dtypes

dtype('O')

In [9]:
#Convert Column to all lower case, strip punctuation marks
df["req"] = df.req.str.replace(',', '')
df["req"] = df.req.str.replace('.', '')
df["req"] = df["req"].str.lower()

C:\Users\schre\AppData\Local\Temp\ipykernel_30856\2796696168.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["req"] = df.req.str.replace('.', '')


In [10]:
df.head()

,dollars_obligated,description_of_requirement,level_2_category,level_3_category,co_bus_size_determination,business_rule_tier,contract_name,req
0,0.0,DHS BULK CLOSE OUT,Capability As a Service,Network,OTHER THAN SMALL BUSINESS,TIER 2,SCHEDULE 70 - INFORMATION TECHNOLOGY,dhs bulk close out capability as a service net...
1,0.0,ITAS SUPPORT SERVICES,IT Professional Services,IT Management,SMALL BUSINESS,TIER 2,SCHEDULE 70 - INFORMATION TECHNOLOGY,itas support services it professional services...
2,0.0,WIRELESS SERVICE,IT Professional Services,Network,OTHER THAN SMALL BUSINESS,TIER 2,SCHEDULE 70 - INFORMATION TECHNOLOGY,wireless service it professional services netw...
3,0.0,PERFORMANCE PERIOD: 10/24/18-09/30/19DOJ FBI ...,Capability As a Service,Network,OTHER THAN SMALL BUSINESS,TIER 2,SCHEDULE 70 - INFORMATION TECHNOLOGY,performance period: 10/24/18-09/30/19doj fbi ...
4,0.0,IGF::OT::IGF VERIZON WIRELESS FY19 RMB/ IOD,Capability As a Service,Network,OTHER THAN SMALL BUSINESS,TIER 2,SCHEDULE 70 - INFORMATION TECHNOLOGY,igf::ot::igf verizon wireless fy19 rmb/ iod ca...


In [11]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer = word_tokenize, stop_words=stop))
    
])

In [12]:
# Assign the column I want to use as req then run it through the pipeline, fit and transform it


req = df['req']
X = pipeline.fit_transform(req)
X.shape

C:\Users\schre\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


(528640, 158501)

In [13]:
X

<528640x158501 sparse matrix of type '<class 'numpy.float64'>'
	with 8500335 stored elements in Compressed Sparse Row format>

In [14]:
svd = TruncatedSVD(n_components=2, n_iter=7, random_state=42)
svd.fit(X)

In [15]:
# manually fit on batches
kmeans = MiniBatchKMeans(n_clusters=10,random_state=0,batch_size=60000)

kmeans = kmeans.partial_fit(X[0:60000,:])
kmeans = kmeans.partial_fit(X[60000:120000,:])
kmeans = kmeans.partial_fit(X[120000:180000,:])
kmeans = kmeans.partial_fit(X[180000:240000,:])
kmeans = kmeans.partial_fit(X[240000:300000,:])
kmeans = kmeans.partial_fit(X[300000:360000,:])
kmeans = kmeans.partial_fit(X[360000:420000,:])
kmeans = kmeans.partial_fit(X[420000:480000,:])
kmeans = kmeans.partial_fit(X[480000:540000,:])

print(datetime.datetime.now() - now)

0:01:29.063973


In [16]:
labels = kmeans.predict(X)
labels

array([4, 5, 4, ..., 5, 5, 5])

In [17]:
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

from collections import Counter
clusters = Counter(labels)

# print(labels)
print(datetime.datetime.now() - now)
print ("----------------------------------------------------------")
# print(clusters)
print ("----------------------------------------------------------")
print('Number of clusters: %d' % n_clusters_)
print("----------------------------------------------------------")
print('Number of noise points: %d' % n_noise_)

0:01:31.598336
----------------------------------------------------------
----------------------------------------------------------
Number of clusters: 10
----------------------------------------------------------
Number of noise points: 0


In [18]:
# Add clusters label to DF
df['clusters'] = labels
df.head()

,dollars_obligated,description_of_requirement,level_2_category,level_3_category,co_bus_size_determination,business_rule_tier,contract_name,req,clusters
0,0.0,DHS BULK CLOSE OUT,Capability As a Service,Network,OTHER THAN SMALL BUSINESS,TIER 2,SCHEDULE 70 - INFORMATION TECHNOLOGY,dhs bulk close out capability as a service net...,4
1,0.0,ITAS SUPPORT SERVICES,IT Professional Services,IT Management,SMALL BUSINESS,TIER 2,SCHEDULE 70 - INFORMATION TECHNOLOGY,itas support services it professional services...,5
2,0.0,WIRELESS SERVICE,IT Professional Services,Network,OTHER THAN SMALL BUSINESS,TIER 2,SCHEDULE 70 - INFORMATION TECHNOLOGY,wireless service it professional services netw...,4
3,0.0,PERFORMANCE PERIOD: 10/24/18-09/30/19DOJ FBI ...,Capability As a Service,Network,OTHER THAN SMALL BUSINESS,TIER 2,SCHEDULE 70 - INFORMATION TECHNOLOGY,performance period: 10/24/18-09/30/19doj fbi ...,5
4,0.0,IGF::OT::IGF VERIZON WIRELESS FY19 RMB/ IOD,Capability As a Service,Network,OTHER THAN SMALL BUSINESS,TIER 2,SCHEDULE 70 - INFORMATION TECHNOLOGY,igf::ot::igf verizon wireless fy19 rmb/ iod ca...,4


In [19]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

labels_color_map = {
    0: '#20b2aa', 1: '#ff7373', 2: '#ffe4e1', 3: '#005073', 4: '#4d0404',
    5: '#ccc0ba', 6: '#4700f9', 7: '#f6f900', 8: '#00f91d', 9: '#da8c49'
}
pca_num_components = 2
tsne_num_components = 2

reduced_data = PCA(n_components=pca_num_components).fit_transform(X)
# print reduced_data

fig, ax = plt.subplots()
for index, instance in enumerate(reduced_data):
    # print instance, index, labels[index]
    pca_comp_1, pca_comp_2 = reduced_data[index]
    color = labels_color_map[labels[index]]
    ax.scatter(pca_comp_1, pca_comp_2, c=color)
plt.show()

TypeError: PCA does not support sparse input. See TruncatedSVD for a possible alternative.

In [26]:
## Plot the data
#plt.scatter(X[:,0], 
#            X[:,1])
#
## Plot the clusters 
#plt.scatter(kmeans.cluster_centers_[:, 0], 
#            kmeans.cluster_centers_[:, 1], 
#            s=200,                             # Set centroid size
#            c='red')                           # Set centroid color
#plt.show()

In [3]:
#from sklearn.metrics.pairwise import pairwise_distances_argmin
#fig = plt.figure(figsize=(8, 3))
#fig.subplots_adjust(left=0.02, right=0.98, bottom=0.05, top=0.9)
#colors = ["#4EACC5", "#FF9C34", "#4E9A06"]
#
## We want to have the same colors for the same cluster from the
## MiniBatchKMeans and the KMeans algorithm. Let's pair the cluster centers per
## closest one.
#k_means_cluster_centers = kmeans.cluster_centers_
#order = pairwise_distances_argmin(X, kmeans.cluster_centers_)
#
#k_means_labels = pairwise_distances_argmin(X, k_means_cluster_centers)
##print(k_means_labels)
#
#plt.show()

In [2]:
#import matplotlib.pyplot as plt
#import seaborn as sns
#sns.set()
#plt.style.use('seaborn-whitegrid')
#plt.rc("figure", autolayout=True)
#plt.rc("axes", labelweight='bold', labelsize='large', titleweight='bold', titlesize=14, titlepad=10)
#sns.relplot(x='longitude', y='latitude', hue='labels', data=df, height=6)
#plt.show()

In [1]:
#plt.scatter(X[:, 0], X[:, 1], c=labels, s=50, cmap='viridis')
#
#centers = kmeans.cluster_centers_
#plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);